## File mounting

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "Uploading_Data_Colab_1.xlsx" with length 9000 bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving qlora_pico_indo_2200plus.jsonl to qlora_pico_indo_2200plus.jsonl
User uploaded file "Uploading_Data_Colab_1.xlsx" with length 9000 bytes


In [ ]:
filepath = "/content/qlora_pico_indo_2200plus.jsonl"

### Opening JSONL file



In [ ]:
import json
from pprint import pprint

In [ ]:
import json
from pprint import pprint

dataList = []
with open(filepath, 'r') as f:
    for line in f:
        try:
            dataList.append(json.loads(line))
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON on line: {line.strip()}\nError: {e}")

pprint(dataList)

Streaming output truncated to the last 5000 lines.
           'subkutikular kontinu: kurangi risiko dehisensi. perawatan lanjut - '
           'buka balutan setelah 12 jam bila luka kering - npwt (negative '
           'pressure wound therapy): percepat penyembuhan luka berat - '
           'silicone sheet: kurangi parut - bukti untuk madu, mitomycin, '
           'imiquimod, bawang masih lemah dan terapi farmakologis sedasi & '
           'analgesia - anestesi lokal: jarum kecil (27g), cryotherapy, atau '
           'anestesi topikal (let/emla) - blok saraf digital: efektif untuk '
           'luka jari - sedasi: pada anak untuk mempercepat prosedur '
           'antibiotik profilaksis - tidak rutin untuk luka bersih & pasien '
           'sehat - diperlukan bila luka kotor, gigitan, crush injury, atau '
           'pasien imun lemah - luka tusuk (punctum), terutama gigitan anjing '
           'wajib antibiotik.\n'
           'C: Tanpa intervensi terarah atau hanya observasi.\n'
     

## Loading the model

### Install packages and necessary independencies

In [ ]:
!pip install -U accelerate bitsandbytes datasets peft transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.7/60.7 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 32.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 59.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 18.0 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
  Attempting uninstall: transformers
    Found existing installation: transformers 5.0.0
    Uninstalling transformers-5.0.0:
      Successfully uninstalled transformers-5.0.0


### Load the model

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer, BitsAndBytesConfig
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

modelPath = "Henrychur/MMed-Llama-3-8B"

model = AutoModelForCausalLM.from_pretrained(
    modelPath,
    device_map="auto",
    quantization_config=BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.bfloat16,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_use_double_quant=True,
    ),
    torch_dtype=torch.bfloat16,
)

print("Setting up tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(modelPath, use_fast=False)

# fix padding token
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

# add special token
tokenizer.add_tokens(["<|im_start|>"])
tokenizer.add_special_tokens(dict(eos_token="<|im_end|>"))

# update model embeddings
model.resize_token_embeddings(len(tokenizer))
model.config.eos_token_id = tokenizer.eos_token_id

# verification
print(f"Pad token: {tokenizer.pad_token}")
print(f"Pad token ID: {tokenizer.pad_token_id}")
print(f"EOS token: {tokenizer.eos_token}")
print(f"EOS token ID: {tokenizer.eos_token_id}")

ValueError: Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit the quantized model. If you want to dispatch the model on the CPU or the disk while keeping these modules in 32-bit, you need to set `llm_int8_enable_fp32_cpu_offload=True` and pass a custom `device_map` to `from_pretrained`. Check https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu for more details. 

### QLoRA initialization

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 41,943,040 || all params: 8,072,220,672 || trainable%: 0.5196


### Format data

In [ ]:
def format_instruction(filepath):
    """
    Formats PICO data into conversation format.
    Your data structure:
    - instruction: Task instruction (how to respond)
    - input: PICO framework information
    - output: Expected clinical reasoning response
    """
    # combine instruction and input
    user_message = f"{filepath['instruction']}\n\n{filepath['input']}"

    # add special tokens to format as convo
    text = f"<|im_start|>user\n{user_message}<|im_end|>\n<|im_start|>assistant\n{filepath['output']}<|im_end|>"

    return {"text": text}

# test the conversation format
print("\n" + "="*500)
print("Example of formatted text:")
print("="*500)
print(format_instruction(dataList[2198])['text'][:100000] + "...")
print("="*500 + "\n")


Example of formatted text:
<|im_start|>user
Fokus pada mekanisme sebab-akibat yang mendasari keputusan klinis.

PICO:
P: Individu dengan faktor risiko  dan keluhan jenis kandidiasis mulut: kandidiasis pseudomembran akut: merupakan bercak putih pseudomembran yang tegas, rapuh, bila dihilangkan, akan meninggalkan jejak bercak berwarna merah. kandidiasis eritematosa akut: bercak merah dan selaput lendir yang mengalami erosi dan atrofi, terutama pada bagian punggung lidah. • kandidosis pseudomembran kronis: secara klinis tidak berbeda dengan jenis pseudomembran akut, tetapi lesi bersifat persisten. • kandidosis eritematosa kronis: radang pada lapisan epitel di daerah bantalan gigi palsu atau anak-anak yang memakai alat ortodontik. • kandidosis hiperplastik kronis: plak yang sangat persisten, keras, tidak teratur, ditemukan di mulut, pipi, dan lidah. • kandidosis nodular kronis: nodul-nodul pada lidah, merupakan bentuk langka • angular cheilitis: rasa nyeri pada sudut mulut yang meluas ke 

### Tokenize dataset

In [ ]:
from datasets import Dataset

In [ ]:
dataset = Dataset.from_list(dataList)
dataset = dataset.map(format_instruction)

def tokenize_function(examples):
    tokenized = tokenizer(
        examples["text"],
        truncation=True,
        max_length=1024,
        padding="max_length",
    )
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names
)

print(f"Tokenized dataset size: {len(tokenized_dataset)}")

# check token length distribution
print("\nChecking token lengths...")
sample_lengths = []
for i in range(min(100, len(dataList))):
    formatted = format_instruction(dataList[i])
    tokens = tokenizer(formatted["text"])["input_ids"]
    sample_lengths.append(len(tokens))

print(f"Average token length: {sum(sample_lengths)/len(sample_lengths):.0f}")
print(f"Max token length in sample: {max(sample_lengths)}")
print(f"Min token length in sample: {min(sample_lengths)}")


Map:   0%|          | 0/2831 [00:00<?, ? examples/s]

Map:   0%|          | 0/2831 [00:00<?, ? examples/s]

TypeError: '<' not supported between instances of 'NoneType' and 'int'

### Training

In [ ]:
training_args = TrainingArguments(
    output_dir="./qlora-medical-output",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    fp16=False,
    bf16=True,
    logging_steps=10,
    save_strategy="epoch",
    save_total_limit=2,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.03,
    gradient_checkpointing=True,
    max_grad_norm=0.3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

### Start training

In [ ]:
print("\nStarting training...")
trainer.train()

### Save the trained model

In [ ]:
model.save_pretrained("./qloraGPT")
tokenizer.save_pretrained("./qloraGPT")
print("\n✓ Done! Adapters saved to ./qloraGPT")